# **Imports**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# **Creating obejctes' information array**

In [ ]:
info = np.zeros((4, 15))

weights   = [14, 15, 12, 10, 18, 14,  2,  5,  10, 1,   8,  14, 15, 16, 19]
prices    = [12, 10, 5,  2,  10, 8,   10, 10, 12, 1,   14, 16, 17, 18, 20]
obj_class = [1,  1,  1,  1,  1,  1,   2,  2,  2,  2,   3,  3,  3,  3,  3 ]

weights   = np.array(weights)
prices    = np.array(prices)
obj_class = np.array(obj_class)

p_to_w_ratios = prices / weights

In [ ]:
info[0] = weights
info[1] = prices
info[2] = obj_class
info[3] = p_to_w_ratios

In [ ]:
sorted_info = info[ :, info[3].argsort()]
sorted_info

array([[10.        , 12.        , 18.        , 14.        , 15.        ,
        14.        ,  1.        , 19.        , 16.        , 15.        ,
        14.        , 10.        ,  8.        ,  5.        ,  2.        ],
       [ 2.        ,  5.        , 10.        ,  8.        , 10.        ,
        12.        ,  1.        , 20.        , 18.        , 17.        ,
        16.        , 12.        , 14.        , 10.        , 10.        ],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  2.        ,  3.        ,  3.        ,  3.        ,
         3.        ,  2.        ,  3.        ,  2.        ,  2.        ],
       [ 0.2       ,  0.41666667,  0.55555556,  0.57142857,  0.66666667,
         0.85714286,  1.        ,  1.05263158,  1.125     ,  1.13333333,
         1.14285714,  1.2       ,  1.75      ,  2.        ,  5.        ]])

# **Implementation of Algorithm**

## **Normaliztion Functions**

These functions check whether the given gen is under problem's limits(For Example, does gen have at leasst an object from every class?) or not. If doesn't, make it under the limits!

Returns the classes are included in gen.

In [ ]:
def included_classes(gen):
  c = []
  for i in range(len(gen)):
    if gen[i] > 0:
      c.append(sorted_info[2, i])
  return set(c)

Returns the classes are not included in gen.

In [ ]:
def not_included_classes(includeds):
  c = [1, 2, 3]
  for i in list(includeds):
    c.remove(i)
  return c

In [ ]:
def calculate_the_value_of_gen(gen):
  gen = gen.reshape((gen.shape[0], 1))

  return np.dot(sorted_info[1], gen)

Calculates the total weight of gen.

In [ ]:
def calculate_cumulative_weights(gen):
  gen = gen.reshape((gen.shape[0], 1))

  return np.dot(sorted_info[0], gen)

If there is not any object from a class, this function add the best and possible object from that class to gen.

In [ ]:
def make_gen_complete(gen, not_includeds):
  for i in range(len(gen) - 1, 0, -1):
    if len(not_includeds) == 0:
      return gen

    if gen[i] == 0:
      if info[2, i] in not_includeds:
        gen[i] += 1
        not_includeds.remove(info[2, i])

  return gen

Makes total weight of gen less than ***300***.

In [ ]:
def make_gen_under_the_limit(gen, weight_limit = 300):
  for i in range(len(gen)):

    for j in range(int(gen[i])):

      if calculate_cumulative_weights(gen) <= weight_limit:
        return gen
      gen[i] -= 1

      if len(included_classes(gen)) < 3:
        gen[i] += 1
        continue
  
  return gen

Ckecks all limits and corrects the gen if it is not.

In [ ]:
def normalize(gen, weight_limit = 300):

  includeds = included_classes(gen)
  if not len(includeds) == 3:
    gen = make_gen_complete(gen, not_included_classes(includeds))

  if calculate_cumulative_weights(gen) > weight_limit:
    gen = make_gen_under_the_limit(gen, weight_limit = 300)

  return gen

Makes the call's value between 0 to 5, only if it is not!

In [ ]:
def make_cell_value_under_the_limits(val, min_val=0, max_val=5):
  if val > max_val or val < min_val:
    val = random.randint(min_val, max_val)
  
  return val

## **Main Part**

This function generates the initial generation in given size.

In [ ]:
def generate_initial_population(size=50, n_features=15):
  population = np.random.randint(5, size=(size, n_features))

  for i in range(size):
    population[i] = normalize(population[i], weight_limit = 300)
    
  return population

This function does a one point crossover.

In [ ]:
def crossover(f_gen, s_gen):
  i = random.randint(0, len(f_gen) - 1)
  new_f = np.append(f_gen[:i], s_gen[i:])
  new_s = np.append(s_gen[:i], f_gen[i:])

  return new_f, new_s

In [ ]:
def indexes_generator(max_ind):
  f = random.randint(0, max_ind)
  s = random.randint(0, max_ind)
  while f == s:
    s = random.randint(0, max_ind)
  return f, s

Does crossover over population!

In [ ]:
def do_crossover(population, number_of_crossover=20, random_selection=5):
  for i in range(number_of_crossover):
    random_selected_gens = np.random.randint(len(population) - 2, size=(random_selection, 1)).tolist()
    fi, si = indexes_generator(len(random_selected_gens) - 2)

    new_f, new_s = crossover(population[random_selected_gens[fi]], population[random_selected_gens[si]])

    population = np.append(population, normalize(new_f).reshape(1, new_f.shape[0]), 0)
    population = np.append(population, normalize(new_s).reshape(1, new_s.shape[0]), 0)

  return population

At first, generates a random number between zero to 1, then compares it with the mutation probability, if it's prossible, generates the sign randomly. At the end, Adds or subtracs a random number between 0 to 2(basen on the generated sign).

In [ ]:
def mutation(gen, probability=0.70, max_limit = 2):
  p = random.uniform(0, 1)
  if p <= probability:
    sign = random.uniform(0, 1)

    i = random.randint(0, len(gen) - 1)
    if sign > 0.5:
      gen[i] += random.randint(0, max_limit)
    else:
      gen[i] -= random.randint(0, max_limit)

    gen[i] = make_cell_value_under_the_limits(gen[i])

  return normalize(gen)

Does mutation over population.

In [ ]:
def do_mutation(population, probability = 0.70):
  for i in range(len(population)):
    population[i] = mutation(population[i], probability)
  
  return population

Fitness function which calculates the total value of price / weight for the given gen.

In [ ]:
def fitness_func(gen):
  return np.dot(gen, sorted_info[3].T)

These two functions sort population based on fitness function.

In [ ]:
def sort_values(population):
  f = fitness_func(population)

  f = f.reshape((f.shape[0], 1))
  pop_fitness = np.append(population, f, 1)
  pop_fitness = pop_fitness[np.argsort(pop_fitness[:, -1])]
  return pop_fitness[:, :-1]

In [ ]:
def selection(population, size=50):
  return population[-size:-1]

My Eveloutionary algorithm!

Represantation: An array of Arithmetic number values.

Crossover: One-point

Mutation: Adding or subtracting a random number(between 0-2) to a random cell under the probability of 0.75.

Selection: Selects the 50 best gens

In [ ]:
def GA(iterations = 1000, pop_size = 50, n_crossover = 20, mutation_prob=0.75, n_features=15):
  population = generate_initial_population(size=pop_size, n_features=n_features)

  for i in range(iterations):
    if i % 10000 == 0:
      print(i)
    new_gens = do_crossover(population, number_of_crossover=20, random_selection=5)
    population = do_mutation(population, probability = 0.70)

    population = np.append(population, new_gens, 0)

    population = sort_values(population)

    population = selection(population, size=pop_size)

  return population[-1]

In [ ]:
best_gen = GA()

0


**Best gen**

In [ ]:
best_gen

array([0., 0., 0., 0., 0., 1., 5., 2., 1., 5., 4., 2., 5., 5., 5.])

**Total weight of Best Gen**

In [ ]:
calculate_cumulative_weights(best_gen)

array([299.])

**Total value of best gen**

In [ ]:
calculate_the_value_of_gen(best_gen)

array([418.])

It's obvious, If we had continued GA for more iterations, we would have gotten a better gen.